# Extract Logic

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import dotenv
import json
import re
from pyspark.sql.functions import regexp_replace, col
import os
os.environ["HADOOP_HOME"] = r"C:\utils\winutils\hadoop-3.0.0"
os.environ["PATH"] += os.pathsep + r"C:\utils\winutils\hadoop-3.0.0\bin"
dotenv.load_dotenv()

conf = SparkConf().setAppName("MOD Extract POC") \
    .set("spark.eventLog.enabled", "false") \
    .set("spark.driver.memory", "6g") \
    .set("spark.executor.memory", "6g") \
    .set("spark.driver.maxResultSize", "6g")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [3]:
with open('extract_query 2.sql', 'r') as sql_file:
    extract_query = sql_file.read()
db_url = "jdbc:postgresql://aapsql-apm1012739-00dev01.cbogo6ayippz.us-east-2.rds.amazonaws.com:5432/postgres"
db_user = "src_mod_dev"
db_password = "{yy92K9Y_4FMZ,01"
db_driver = "org.postgresql.Driver"

In [4]:
df = (
        spark.read.format("jdbc")
        .option("url", db_url)
        .option("user", db_user)
        .option("password", db_password)
        .option("query", extract_query)
        .option("driver", "org.postgresql.Driver")
        .load()
    )

df.printSchema()
df.createOrReplaceTempView("MOD_POC_MEM_LD")

input_df = spark.sql("Select * from MOD_POC_MEM_LD")
print("Input Count")
input_df.count()

input_df.select("Member ID").show(200, truncate=False)

root
 |-- Member ID: string (nullable = true)
 |-- FIRST NAME: string (nullable = true)
 |-- LAST NAME: string (nullable = true)
 |-- ADDRESS STREET 1: string (nullable = true)
 |-- ADDRESS STREET 2: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZIP Code: string (nullable = true)
 |-- Email Address: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- Number of Members: integer (nullable = true)
 |-- Plan State: string (nullable = true)
 |-- Spoken Language: string (nullable = true)

Input Count
+--------------------+
|Member ID           |
+--------------------+
|974W29202           |
|974Y10226           |
|974Y10280           |
|974Y13481           |
|974Y19135           |
|974Y23553           |
|975A66452           |
|975M88370           |
|975W11899           |
|975W12965           |
|975W14094           |
|975W21132         

In [ ]:
with open("./rules 3.json", "r") as file:
    rules_dict = json.load(file)
print("Rules Dictionary: ", rules_dict)
rules = sorted(rules_dict.get("rules", []), key=lambda x: x.get('executionOrder', float('inf')))
print("Rules: ", rules)
output_config = rules_dict.get("outputConfig", {})
print("Output Config: ", output_config)
def fix_query_for_spark(query):
    # Replace "column" with `column`
    return re.sub(r'"([^"]+)"', r'`\1`', query)


Rules Dictionary:  {'rules': [{'ruleName': 'BIRTH DATE Format Transformation', 'ruleId': '1', 'executionOrder': 1, 'rulesData': {'expression': "SELECT {cols},CASE WHEN LENGTH({cond_col1}) = 8 THEN CONCAT(SUBSTRING({cond_col1}, 5, 2), '/', SUBSTRING({cond_col1}, 7, 2), '/', SUBSTRING({cond_col1}, 1, 4)) ELSE {update_col1} END AS {update_col1} FROM MOD_POC_MEM_LD", 'params': {'update_cols': {'update_col1': 'Birth_Date'}, 'cond_cols': {'cond_col1': 'Birth_Date'}}}}, {'ruleName': 'Output file Structure', 'ruleId': '2', 'executionOrder': 2, 'rulesData': {'expression': 'SELECT "Member ID", "First Name", "Last Name", "Address 1", "Address 2", "City", "State", "ZIP Code", "Email Address", "Phone Number", Birth_Date as "Birth Date", "Gender", "Number of Members", "Plan State", "Spoken Language" FROM MOD_POC_MEM_LD', 'params': {'update_cols': {}, 'cond_cols': {}}}}]}
Rules:  [{'ruleName': 'BIRTH DATE Format Transformation', 'ruleId': '1', 'executionOrder': 1, 'rulesData': {'expression': "SELECT 

In [6]:


for rule in rules:
    cols_df = spark.sql("SHOW COLUMNS IN MOD_POC_MEM_LD").collect()
    cols = [col.col_name for col in cols_df]
    print("Columns in standard_layout: ", cols)
    query = rule.get('rulesData', {}).get('expression', '')
    params = rule.get('rulesData', {}).get('params', {})
    update_cols = params.get('update_cols', {})
    cond_cols = params.get('cond_cols', {})
    view = "MOD_POC_MEM_LD"
    required_cols = ", ".join([f"`{col}`" for col in cols if col not in update_cols.values()])
    print(f"Required Columns: {required_cols}")
    print(f"keys: {update_cols.keys()}\nvalues: {update_cols.values()}")
    params = list(zip(list(update_cols.keys()), list(update_cols.values())))
    if cond_cols:
        cond_params = list(zip(list(cond_cols.keys()), list(cond_cols.values())))
        params = params + cond_params
    print(f"zip: {params}")
    for i, j in params:
        query = query.replace(f"{{{i}}}", f"`{j}`")
    query = query.replace("{cols}", required_cols)
    expression = fix_query_for_spark(query)
    
    print(expression)

    # condition = rule.get('condition', '')
    # if condition:z
    #     query = f"SELECT {source} FROM {target} WHERE {condition}"
    # else:
    #     query = f"SELECT {source} FROM {target}"
    result_df = spark.sql(expression)
    result_df.show()
    result_df.createOrReplaceTempView(view)
    # result_df.show()
    print(view)
    # result_df.printSchema()


out_df = spark.sql("SELECT * FROM MOD_POC_MEM_LD")
# out_df.filter("case_num = 'CA458C'").show()
for c in out_df.columns:
    out_df = out_df.withColumn(c, regexp_replace(col(c), r'^\s*$', ''))
out_df = out_df.na.replace(['', '0000000000    '], [None, None])
out_df.printSchema()
out_df.show(100, truncate=False)

# out_df.coalesce(1).write.option("delimiter", "|").option("header", True).csv("output_csv")

Columns in standard_layout:  ['Member ID', 'FIRST NAME', 'LAST NAME', 'ADDRESS STREET 1', 'ADDRESS STREET 2', 'City', 'State', 'ZIP Code', 'Email Address', 'Phone Number', 'Birth Date', 'GENDER', 'Number of Members', 'Plan State', 'Spoken Language']
Required Columns: `Member ID`, `FIRST NAME`, `LAST NAME`, `ADDRESS STREET 1`, `ADDRESS STREET 2`, `City`, `State`, `ZIP Code`, `Email Address`, `Phone Number`, `Birth Date`, `GENDER`, `Number of Members`, `Plan State`, `Spoken Language`
keys: dict_keys(['update_col1'])
values: dict_values(['Birth_Date'])
zip: [('update_col1', 'Birth_Date'), ('cond_col1', 'Birth_Date')]
SELECT `Member ID`, `FIRST NAME`, `LAST NAME`, `ADDRESS STREET 1`, `ADDRESS STREET 2`, `City`, `State`, `ZIP Code`, `Email Address`, `Phone Number`, `Birth Date`, `GENDER`, `Number of Members`, `Plan State`, `Spoken Language`,CASE WHEN LENGTH(`Birth_Date`) = 8 THEN CONCAT(SUBSTRING(`Birth_Date`, 5, 2), '/', SUBSTRING(`Birth_Date`, 7, 2), '/', SUBSTRING(`Birth_Date`, 1, 4)) EL

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Birth_Date` cannot be resolved. Did you mean one of the following? [`Birth Date`, `State`, `City`, `GENDER`, `Plan State`].; line 1 pos 243;
'Project [Member ID#0, FIRST NAME#1, LAST NAME#2, ADDRESS STREET 1#3, ADDRESS STREET 2#4, City#5, State#6, ZIP Code#7, Email Address#8, Phone Number#9, Birth Date#10, GENDER#11, Number of Members#12, Plan State#13, Spoken Language#14, CASE WHEN ('LENGTH('Birth_Date) = 8) THEN 'CONCAT('SUBSTRING('Birth_Date, 5, 2), /, 'SUBSTRING('Birth_Date, 7, 2), /, 'SUBSTRING('Birth_Date, 1, 4)) ELSE 'Birth_Date END AS Birth_Date#73]
+- SubqueryAlias mod_poc_mem_ld
   +- View (`MOD_POC_MEM_LD`, [Member ID#0,FIRST NAME#1,LAST NAME#2,ADDRESS STREET 1#3,ADDRESS STREET 2#4,City#5,State#6,ZIP Code#7,Email Address#8,Phone Number#9,Birth Date#10,GENDER#11,Number of Members#12,Plan State#13,Spoken Language#14])
      +- Relation [Member ID#0,FIRST NAME#1,LAST NAME#2,ADDRESS 

In [ ]:
print("Output count ")
out_df.count()

Output count 


187

In [ ]:
out_df.show(out_df.count(), truncate=False)

+--------------------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+--------------------------------------------------+---------+-------------+-----+--------+------------------------------------------------------------------------------------------------------------------------+--------------+----------+------+-----------------+----------+---------------+
|Member ID           |First Name                                                                      |Last Name                                                                       |Address 1                                         |Address 2|City         |State|ZIP Code|Email Address                                                                                                           |Phone Number  |Birth Date|Gender|Number of Members|Plan State|Spoken Language|
+--------------------+------------------------------